# Improving the Detection of Small Oriented Objects in Aerial Images

[Original Paper](https://arxiv.org/pdf/2401.06503v1.pdf)

## Abstract

The paper introduces a method for accurately detecting small oriented objects in aerial images, addressing challenges related to their size and orientation. The proposed Attention-Points Network incorporates two losses, namely Guided-Attention Loss (GALoss) and Box-Points Loss (BPLoss). GALoss utilizes an instance segmentation mask to enhance attention features crucial for detecting small objects. These attention features contribute to predicting box points in BPLoss, determining their position relative to the target oriented bounding box. Experimental results on datasets such as DOTA-v1.5 and HRSC2016 demonstrate the effectiveness of the proposed approach.

## 1. Introduction

This research addresses the specific challenge of accurately detecting small oriented objects in aerial images, a task not effectively handled by conventional horizontal bounding boxes (HBB). The paper argues that employing HBBs for oriented aerial objects is inefficient, leading to imprecise localization and increased noise. The proposed solution, the Attention-Points Network, introduces two critical losses to enhance the model's capabilities in detecting small oriented objects.

The first loss, Guided-Attention Loss (GALoss), is designed to leverage attention features obtained from an instance segmentation mask. This is aimed at improving the model's detection performance, especially concerning challenging instances such as small and complex objects. To overcome the difficulty in annotating irregularly shaped aerial objects, coarse-level masks requiring only bounding box coordinates are utilized.

The second loss, Box-Points Loss (BPLoss), utilizes attention features to predict box points, which are then scored based on their relative position to the target oriented bounding box (OBB). The paper emphasizes that these losses collectively contribute to refining the model's awareness and improving its detection accuracy for small oriented objects.

The experimental evaluation is conducted on the DOTA-v1.5 dataset, focusing on instances smaller than 10 pixels. The proposed Attention-Points Network is compared against existing oriented bounding box (OBB) detectors, showcasing its effectiveness not only on the standard DOTA-v1.5 dataset but also on a maritime-related dataset (HRSC2016). Additionally, ablation experiments confirm the positive impact of GALoss and BPLoss on overall performance, establishing their significance in the proposed framework.

## 2. Related work

In this section, the paper discusses approaches to object detection using bounding boxes, focusing on proposal generation and regression loss design. The context extends to oriented object detection methods and explores the incorporation of attention mechanisms for detecting objects in aerial images. This overview sets the stage for introducing the Attention-Points Network in subsequent sections.

### 2.1. Generating Region Proposals

Generating region proposals involves employing an additional network to predict object locations and classes. In a segmentation-based approach, a segmentation map discriminates between objects and the background, filtering out overlapping proposals with low objectness scores. The IoU between ground-truth and predictions is computed at varying thresholds, typically considering IoU≥0.5 as an object and IoU<0.5 as the background. Subsequently, a convolutional neural network (CNN) handles object classification and localization.

To address proposal generation challenges, recent multi-stage methods have emerged. A region proposal network is designed to optimize cost by sharing convolution layers with the feature extractor. This network generates sets of Regions of Interest (RoIs) by guiding the model on where to focus. By scanning locations in the extracted features, it assesses if further processing is necessary in a particular region. The network employs anchor boxes with two scores indicating the presence or absence of an object at each location.

#### 2.1.1 Oriented Proposals

To represent object rotation in detection, anchors incorporating an angle parameter have been utilized in various methods. Early methods employed fifty-four anchors with different scales, ratios, and angles, achieving good performance in detecting arbitrarily oriented objects. However, the high number of anchors introduced computational complexity and memory overhead. An alternative approach transformed horizontal to rotated Regions of Interest (RoIs) to reduce the anchor count. Yet, the transformation network was complex due to fully connected layers and alignment operations.

Another strategy, leveraging a rotation equivariant feature extractor within a transformation network, aimed to extract rotation-invariant features for region proposals. While it aligned rotated RoIs in their correct orientation dimension, it did not alleviate the computational complexity of the transformation network. Addressing this bottleneck requires designing a more efficient architecture. This motivation led to improvements in previous oriented object detectors, with a focus on efficient proposal generation and object detection. In this work, the network was chosen for its efficient architecture in proposal generation, further enhanced through loss function design.

### 2.2. Regression Loss Design

Regression loss for predicting bounding boxes involves calculating the error between the predicted and ground-truth bounding boxes. There are two main categories of regression losses: L1-type and IoU-based loss. An example of an L1-type loss is the smooth L1 loss, also known as Huber loss, defined as follows:

$loss(x, y) = \begin{cases}
0.5(x - y)^2 & \text{if } |x - y| < 1 \\
|x - y| - 0.5 & \text{otherwise}
\end{cases}$

The smooth L1 loss is less sensitive to outliers, preventing exploding gradients. It avoids squaring the loss when the absolute value is greater than one, preventing high-value losses. However, this loss is not directly correlated with the metric used in object detection.
On the other hand, the IoU-based loss aims to combine regression loss with the metric. It is given by:

$L_{IoU} = 1 - IoU$

While simple to use, the IoU cannot be computed when there is no overlapping area between two bounding boxes. Moreover, it is unsuitable for oriented object detection, as it leads to an undifferentiable function, hindering the backpropagation of gradients for network training.

#### 2.2.1 Oriented IoU-based Loss

In oriented object detection for aerial images, three IoU-based loss strategies are recognized. The initial method computes the polygon distance between the ground-truth and prediction, combining it with smooth L1 loss to address the non-differentiability issue of IoU-based loss. However, due to the undifferentiable nature of IoU-based loss, the gradient direction is largely influenced by smooth L1 loss, resulting in inconsistent metrics. The second approach involves transforming ground-truth and prediction boxes into a 2-D Gaussian distribution and employing Wasserstein distance and Kullback-Leibler divergence for loss calculation. While this approximation makes the IoU-based loss differentiable, it introduces complexity through the conversion and distance calculations, imposing significant overhead on the network. The third method directly computes IoU-based loss by aggregating the contribution of overlapping pixels between ground-truth and prediction boxes. This approach utilizes the normal distance between pixels and the oriented bounding box (OBB) center but falls short in accurately representing the importance of each pixel, treating them with uniform attention. In this paper, an IoU-based loss is devised by predicting box-points from attention features, and these box-points are evaluated based on their relative position to the target OBB.

### 2.3. Attention Mechanism

The convolutional neural network (CNN) functions as an attention mechanism in computer vision, employing filters to process input features and introducing non-linearity through activation functions. An example of an object detection model utilizing CNN as an attention mechanism is found in Mask-guided attention network for occluded pedestrian detection., where the focus is on enhancing the detection of occluded objects identified by the region proposal network.

However, a drawback of using CNN as an attention mechanism lies in the limited filter sizes, typically constrained to 3x3 or 5x5. As the filter size increases, attention features become coarser, resulting in CNNs capturing attention primarily in the local space of a filter and neglecting farther features.

#### 2.3.1 Self-Attention

Self-attention, initially applied in natural language processing (NLP) to address challenges in understanding lengthy sentences, comprises Queries, Keys, and Values. This mechanism transforms input sentences into vectors, calculates attention scores, and emphasizes relevant words for prediction. Initially prominent in NLP, self-attention, as seen in the Vision Transformer (ViT) architecture, has gained traction in computer vision, sometimes even replacing CNNs in image classification tasks. However, its global computation poses computational challenges compared to localized CNNs.

To address this, a proposal was made to swap the positions of Queries and Values, maintaining the essence of self-attention but altering the use of dot-products, thereby reducing computational complexity. Utilizing the efficiency of self-attention, we integrated this concept to capture global features for detecting small oriented objects in aerial images. Additionally, a loss function was devised to refine attention features by comparing them to object segmentation masks

## 3. Attention-Points Network

We present the details of our proposed small oriented object detector with Attention-Points Network (APN) featuring two new loss functions: Guided-Attention Loss (GALoss) and Box-Points Loss (BPLoss). The baseline is derived from Oriented RCNN and APN is placed after the rotated RoIAlign. This two-stage detector consists of feature extraction in the first stage and prediction in the second stage. ResNet serves as the backbone, producing five levels of features, and these features are input to the Feature Pyramid Network (FPN) for refined feature extraction. The features are then used by the Region Proposal Network (RPN) to generate proposals of various scales and ratios, indicating potential object locations. These proposals are transformed into features through the rotated RoIAlign operation and used as input to the self-attention as region-of-interests (RoI).

For the self-attention input, each RoI has a feature size of 7x7, refined using GALoss. The attention features predict box-points, and their scoring is based on the relative position to the target Oriented Bounding Box (OBB) using BPLoss. The overall architecture is depicted in Figure 1.

<img src="./Summary_IMG/Architecture of Attention-Points Network.PNG" width="900"> 

### 3.1. Guided-Attention Loss ( GALoss )

To ensure effective object highlighting within each Region of Interest (RoI), a loss function is employed to compare attention features with object masks. These masks are derived by converting the bounding box into instance segmentation of the object. Inspired by a concept presented in previous work Mask-guided attention network for occluded pedestrian detection, we utilized self-attention instead of CNN to generate attention features, capturing the global context of objects within the RoIs.

In this process, RoIs undergo self-attention to acquire comprehensive attention features. Subsequently, binary cross-entropy is applied to assess the similarity between the obtained features and masks. As the model undergoes training, attention features learn to correlate with object masks, focusing on the foreground. This results in richer information than the original RoI, contributing to improved small object detection in aerial images and enhanced performance on complex instances. The algorithmic representation of the Guided-Attention Loss (GALoss) is provided in Algorithm 1 and illustrated in Figure 2.

<img src="./Summary_IMG/Box_point.PNG" width="500"> 

### 3.2. Box-Points Loss ( BPLoss )

BPLoss is a function designed to calculate the distance between an Oriented Bounding Box (OBB) and a box-point located at coordinates (i, j), as depicted in Figure 3. The determination of the relative position of a box-point, whether it is inside or outside the box, is computed through the following steps:

$\delta(\text{BP}_i,\text{j}|\text{OBB}) = \begin{cases}
1, & \sum_{n=1}^{P_4} \text{Area}(\text{T}_n) \le \text{Area}(\text{OBB}) \\
0, & \text{otherwise}
\end{cases}$
$(4)$

As indicated in equation (4), BP represents the box-point, and AreaOBB denotes the area of the Oriented Bounding Box (OBB). The condition stated in (4) checks whether the sum of the areas of the triangles is less than or equal to the AreaOBB. If this condition is met, it signifies that the box-point is inside the OBB; otherwise, it is outside. However, since equation (4) is non-differentiable, making it impractical for training due to the lack of useful gradients, we have introduced a kernel function to approximate (4), represented by:

$\delta(\text{BP}_i,\text{j}|\text{OBB}) = \frac{2}{1 + \text{ek}} \sum_{n=1}^{P_4} \frac{\text{Area}(\text{T}_n) - \text{Area}(\text{OBB})}{\text{Area}(\text{OBB})}$
$(5)$

Ultimately, to calculate the Box-Points (BP) Loss, we subtract one from the sum of contributions from each kernel, averaged by the total number of points. This is expressed by equation (6).

$BP\text{ Loss} = 1 - \frac{\sum_{n=1}^N \delta(\text{BP}_n|\text{OBB})}{N}$ 
$(6)$

The computation of BPLoss shares similarities with the PIoU Loss [6]. In PIoU Loss, the distance between pixels and the OBB center is computed, utilizing the OBBs of both the target and prediction. In contrast, BPLoss only employs the target OBB, avoiding the need to convert coordinates into bounding box center format, as we solely require the vertices. Additionally, our distance calculation between the box-points and OBB involves the difference of areas, which differs from PIoU Loss that utilizes the Euclidean distance of pixels.

### 3.3. Evaluation

To assess the performance of our small oriented object detector, we employ the mean average precision (mAP) as the evaluation metric. The mAP is computed by considering the area of intersection over the area of union (IoU) between the ground-truth and predicted boxes. Different IoU thresholds are set to determine true positives (TP) and false positives (FP) based on the level of overlap. Precision scores are then calculated, representing the ratio of TP to the sum of all positive predictions (TP + FP). The average precision (AP) for each class (i) is determined at various IoU thresholds, and the mAP is obtained by averaging AP across all classes.

$mAP = 1/N \sum_{i=1}^N AP_i$

Our evaluation utilizes the PASCAL VOC 2007 (VOC07) [9] and 2012 (VOC12) [10] mAP metrics, defaulting to IoU=0.5 for VOC07. We also explore alternative metrics such as (VOC12) and IoU thresholds commonly accepted in standards, including IoU=0.7 and IoU=0.5:0.95.

## 4. Experiments

In our evaluation process, we employed two distinct datasets: DOTA-v1.5 , which serves as a benchmark for standard aerial images featuring small object instances, and HRSC2016, specifically chosen for its focus on maritime-related images.

### 4.1. Datasets

The Dataset for Object Detection in Aerial Images version-1.5 (DOTA-v1.5) stands out as the most extensive dataset for object detection in aerial images, featuring annotations for oriented bounding boxes. Comprising 2806 large-size images, with splits of 1/2 for training, 1/6 for validation, and 1/3 for testing, DOTA-v1.5 encompasses 403,318 instances across 16 categories. These categories include Plane (PL), Baseball diamond (BD), Bridge (BR), Ground track field (GTF), Small vehicle (SV), Large vehicle (LV), Ship (SH), Tennis court (TC), Basketball court (BC), Storage tank (ST), Soccer-ball field (SBF), Roundabout (RA), Harbor (HA), Swimming pool (SP), Helicopter (HC), and Container Crane (CC). Notably, DOTA-v1.5 introduces the challenge of extremely small instances, some less than 10 pixels, exhibiting significant variations in scale, orientation, and aspect ratio.

The High-Resolution Ship Collections 2016 (HRSC2016) represent a maritime-oriented dataset featuring ships from the sea and inshore environments. Consisting of 1061 images with dimensions ranging from 300×300 to 1500×900 pixels, the dataset includes 436 images for training, 181 for validation, and 444 for testing. For our training purposes, we combined the train and val sets, reserving the test set for evaluation.

### 4.2. Implementation

We conducted model training using a Quadro RTX 8000 and the OBBDetection library, a PyTorch framework featuring various oriented object detection models adapted from the MMdetection toolbox. Comparisons were drawn with results published in the AerialDetection and AlphaRotate libraries.

For the DOTA-v1.5 dataset, we utilized 1024x1024 patches with a 524-stride, and images were resized to scales of 0.5x, 1.0x, and 1.5x with random rotations up to 90 degrees. Training employed the SGD algorithm with a momentum of 0.9 and weight decay of 0.0001. Two dataset splits, trainval/test and train/test, were used for training and evaluation. The former underwent 36 epochs with an initial learning rate of 0.005, with learning rate scheduling at epochs 24 and 33. The latter was trained for 20 epochs with no learning rate scheduling.

For the HRSC2016 dataset, objects were randomly rotated from 0 to 90 degrees during training. Image resizing was set to 1333x800 dimensions, and the model was trained for 180 epochs with R-50-FPN as the backbone.

### 4.3. Comparison with other Methods

On DOTA-v1.5, our method was compared with other approaches. In Table 1, a marginal $mAP_{50}$ increase over the baseline was observed at the trainval/test split. However, a significant improvement of 7.5% was achieved when using the train/test split. Notably, for classes with the smallest instances, small vehicles and ships, our method demonstrated increases of 11.01% and 1.3%, respectively. The variation in performance between trainval/test and train/test splits was attributed to the more complex instances in the validation set. Comparisons were made using the baseline on train/test, as related works only provided $mAP_{50}$ results on trainval/test.

Table 1 illustrated substantial performance gains of our method compared to the baseline on both DOTA-v1.5 trainval/test and train/test splits across $mAP_{75}$ and $mAP_{50:95}$ evaluation metrics. The difference in performance between $mAP_{75}$ and $mAP_{50:95}$ was attributed to the nature of small oriented aerial objects, where using IoU=0.5 could lead to coarse thresholds, potentially missing small instances. Finer thresholds, such as IoU=0.75 and IoU=0.5:0.95, were considered more appropriate. 

For HRSC2016, our method was compared to the baseline and other methods, as shown in Table 3. The comparison utilized $mAP_{50}$ metrics from PASCAL VOC 2007 and VOC 2012. Our method outperformed the baseline in both $mAP_{50}$(07) and $mAP_{50}$(12). Visualization of results on HRSC is presented in Figure 5.

<img src="./Summary_IMG/Tables.PNG" width="1100"> 

### 4.4. Loss Functions

To demonstrate the learning process of our loss functions during training, we generated learning curves over the number of iterations. In Figure 6, we depicted that both GALoss and BPLoss exhibit a decreasing trend. It's essential to note that the plots may appear noisy, as the ground-truths used for loss calculation are coarse-level, such as the object masks and target OBB. It's important to clarify that the figure is not a comparative analysis of the contributions of each loss function but serves as a visualization of how both loss functions learned during the training process.

<img src="./Summary_IMG/GALoss_BPLoss.PNG" width="500">

### 4.5. Ablation Study

To assess the effectiveness of each loss function, we conducted an ablation experiment on the DOTA-v1.5 dataset, comparing performance against the baseline using $mAP_{75}$ and $mAP_{50:95}$ evaluation metrics (Table 4). The experiment involved evaluating two separate cases (Case 1 and Case 2) to measure their individual contributions. The results indicate that both cases contributed positively to the overall performance, without falling behind the baseline. Notably, $mAP_{50:95}$ in Case 3 didn't sum up when adding the results of Case 1 and Case 2. This discrepancy arises because $mAP_{50:95}$ calculates an average performance across IoU thresholds from 0.5 to 0.95, and the coarse threshold of IoU=0.5 may not be suitable for small instances, impacting the calculation of $mAP_{50:95}$.

Furthermore, examining the relative increase in performance for Case 1 and Case 2 and taking the mean, Case 3 shows an above-average result. This observation suggests that our designed loss functions are individually effective within the overall architecture of the Attention-Points Network.

## 5. Conclusion

We introduced the Attention-Points Network and formulated two specialized loss functions, Guided-Attention Loss (GALoss) and Box-Points Loss (BPLoss), tailored for the detection of small oriented objects in aerial images. Our results demonstrated the superior performance of our method compared to the baseline and other architectures. This evaluation was conducted on a standard oriented aerial dataset featuring small object instances (DOTA-v1.5) and a maritime-related dataset (HRSC2016). Additionally, ablation experiments and learning curves for GALoss and BPLoss were provided, validating the efficacy of our proposed method.